In [2]:
pip install json numpy pillow tqdm


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install ultralytics


  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl.metadata (11 kB)
  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached scipy-1.13.1-cp39-cp39-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.7.1-cp39-cp39-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp39-cp39-win_amd64.whl.metadata (6.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.5-cp39-cp39-win_amd64.whl.metadata (109 kB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl.metadata (6.4 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cach

In [ ]:
pip install pandas


In [ ]:
pip install torch torchvision torchaudio


📌 1️⃣ Importing Required Libraries


In [4]:
!pip install pillow

  Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl.metadata (9.2 kB)
Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl (7.0 MB)


In [1]:
# For file and folder operations
import os
import shutil
import json
import numpy as np
from PIL import Image, ImageDraw
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2

📁 2️⃣ Define Dataset Paths

In [5]:
dataset_root = r'D:\Houssam\ObjectDetection'

# For images (leftImg8bit)
images_dir = os.path.join(dataset_root, 'leftImg8bit_trainvaltest', 'leftImg8bit')

# For annotations (gtFine)
annotations_dir = os.path.join(dataset_root, 'gtFine_trainvaltest', 'gtFine')


print("Images folder exists:", os.path.exists(images_dir))
print("Annotations folder exists:", os.path.exists(annotations_dir))

# To see subfolders (like city names)
print("Image subfolders:", os.listdir(images_dir))
print("Annotation subfolders:", os.listdir(annotations_dir))



Images folder exists: True
Annotations folder exists: True
Image subfolders: ['test', 'train', 'val']
Annotation subfolders: ['test', 'train', 'val']


🏷️ 3️⃣ Define Classes and Create Output Label Folder

In [ ]:
    # Output directory for YOLO labels
output_labels_dir = os.path.join(dataset_root, 'cityscapes_yolo', 'labels')
os.makedirs(output_labels_dir, exist_ok=True)

    # Cityscapes classes to YOLO ID mapping
cityscapes_classes = {
        "person": 0,
        "rider": 1,
        "car": 2,
        "truck": 3,
        "bus": 4,
        "motorcycle": 5,
        "bicycle": 6,
        "train": 7,
        "traffic light": 8,
        "traffic sign": 9
}


📐 4️⃣ Convert Polygon to YOLO Bounding Boxes

In [7]:
def convert_polygon_to_bbox(polygon):
    """Convert polygon points to bounding box [x_min, y_min, x_max, y_max]."""
    x_coords = [point[0] for point in polygon]
    y_coords = [point[1] for point in polygon]
    return min(x_coords), min(y_coords), max(x_coords), max(y_coords)

🏗️ 5️⃣ Generate YOLO Labels from Cityscapes JSON

In [8]:
# Loop through both 'train' and 'val'
for split in ['train', 'val']:
    annotations_split_dir = os.path.join(annotations_dir, split)
    images_split_dir = os.path.join(images_dir, split)
    output_labels_split_dir = os.path.join(output_labels_dir, split)
    os.makedirs(output_labels_split_dir, exist_ok=True)

    # Traverse all city folders in this split
    for city in os.listdir(annotations_split_dir):
        city_ann_path = os.path.join(annotations_split_dir, city)
        city_img_path = os.path.join(images_split_dir, city)

        for filename in os.listdir(city_ann_path):
            if filename.endswith("_gtFine_polygons.json"):
                json_path = os.path.join(city_ann_path, filename)
                base_name = filename.replace("_gtFine_polygons.json", "_leftImg8bit.png")
                image_path = os.path.join(city_img_path, base_name)

                if not os.path.exists(image_path):
                    print(f"⚠️ Image not found for {filename}, skipping.")
                    continue

                image = Image.open(image_path)
                img_width, img_height = image.size

                with open(json_path, 'r') as f:
                    data = json.load(f)

                label_lines = []

                for obj in data['objects']:
                    label = obj['label']

                    if label not in cityscapes_classes:
                        continue

                    polygon = obj['polygon']
                    x_min, y_min, x_max, y_max = convert_polygon_to_bbox(polygon)

                    # Normalize for YOLO format
                    x_center = (x_min + x_max) / 2.0 / img_width
                    y_center = (y_min + y_max) / 2.0 / img_height
                    width = (x_max - x_min) / img_width
                    height = (y_max - y_min) / img_height

                    class_id = cityscapes_classes[label]
                    label_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"
                    label_lines.append(label_line)

                # Save labels to file
                if label_lines:
                    label_filename = base_name.replace(".png", ".txt")
                    label_path = os.path.join(output_labels_split_dir, label_filename)

                    with open(label_path, 'w') as label_file:
                        label_file.write("\n".join(label_lines))

print("✅ YOLO label files generated for train and val sets.")

✅ YOLO label files generated for train and val sets.


🧪 6️⃣ Visualize YOLO Labels on Images

In [9]:

id_to_label = {v: k for k, v in cityscapes_classes.items()}

# Draw bounding boxes from YOLO format on an image
def draw_yolo_bboxes(image_path, label_path, id_to_label):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    img_width, img_height = image.size

    if not os.path.exists(label_path):
        print(f"Label file not found: {label_path}")
        return

    with open(label_path, 'r') as f:
        for line in f.readlines():
            parts = line.strip().split()
            class_id, x_center, y_center, width, height = map(float, parts)

            # Convert YOLO format to pixel coordinates
            x_min = (x_center - width / 2) * img_width
            x_max = (x_center + width / 2) * img_width
            y_min = (y_center - height / 2) * img_height
            y_max = (y_center + height / 2) * img_height

            # Draw rectangle
            draw.rectangle([x_min, y_min, x_max, y_max], outline='red', width=2)

            # Get class name
            class_name = id_to_label.get(int(class_id), "unknown")

            # Draw text (class name instead of ID)
            draw.text((x_min, y_min - 10), class_name, fill='red')

    image.save("output_with_boxes.png")
    print("✅ Image saved as output_with_boxes.png")

for split in ['train', 'val']:
    split_dir = os.path.join(images_dir, split)
    for city in os.listdir(split_dir):
        city_img_path = os.path.join(split_dir, city)
        for file in os.listdir(city_img_path):
            if file.endswith('.png'):
                sample_image_path = os.path.join(city_img_path, file)
                sample_label_path = os.path.join(output_labels_dir, split, file.replace('.png', '.txt'))

                if os.path.exists(sample_label_path):
                    print(f"📸 Showing: {sample_image_path}")
                    draw_yolo_bboxes(sample_image_path, sample_label_path, id_to_label)
                    break
        break
    break


📸 Showing: D:\Houssam\ObjectDetection\leftImg8bit_trainvaltest\leftImg8bit\train\aachen\aachen_000000_000019_leftImg8bit.png
✅ Image saved as output_with_boxes.png


📂 7️⃣ Copy Images and Matched YOLO Labels into Final YOLO Folder Structure

In [10]:
# Paths
leftImg8bit_dir = r'D:\Houssam\ObjectDetection\leftImg8bit_trainvaltest\leftImg8bit'
labels_dir = r'D:\Houssam\ObjectDetection\labels'
output_dir = r'D:\Houssam\ObjectDetection\cityscapes_yolo'

# Create YOLO folder structure
for split in ['train', 'val']:
    os.makedirs(os.path.join(output_dir, 'images', split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'labels', split), exist_ok=True)

    split_img_dir = os.path.join(leftImg8bit_dir, split)
    
    for city in os.listdir(split_img_dir):
        city_path = os.path.join(split_img_dir, city)

        # Skip non-directories
        if not os.path.isdir(city_path):
            continue

        for file in os.listdir(city_path):
            if file.endswith('.png'):
                # Copy image
                src_img_path = os.path.join(city_path, file)
                dst_img_path = os.path.join(output_dir, 'images', split, file)
                shutil.copy2(src_img_path, dst_img_path)

                # Copy matching label if it exists
                label_file = file.replace('.png', '.txt')
                src_label_path = os.path.join(labels_dir, label_file)
                dst_label_path = os.path.join(output_dir, 'labels', split, label_file)

                if os.path.exists(src_label_path):
                    shutil.copy2(src_label_path, dst_label_path)
                else:
                    print(f"⚠️ Label not found for {file}, skipping label.")

⚠️ Label not found for strasbourg_000000_036016_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_000294_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_000576_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_001016_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_001236_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_001751_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_002196_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_002963_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_003025_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_003357_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_003920_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000_004617_leftImg8bit.png, skipping label.
⚠️ Label not found for frankfurt_000000

🗂️ 8️⃣ Create data.yaml for Training

In [11]:
data_yaml_content = f"""
train: {os.path.join(output_dir, 'images', 'train').replace(os.sep, '/')}
val: {os.path.join(output_dir, 'images', 'val').replace(os.sep, '/')}

nc: {len(cityscapes_classes)}
names: {list(cityscapes_classes.keys())}
"""

data_yaml_path = os.path.join(dataset_root, 'cityscapes_yolo', 'data.yaml')

with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content.strip())

print(f"✅ data.yaml created at: {data_yaml_path}")
print("Contents:")
print(data_yaml_content)

✅ data.yaml created at: D:\Houssam\ObjectDetection\cityscapes_yolo\data.yaml
Contents:

train: D:/Houssam/ObjectDetection/cityscapes_yolo/images/train
val: D:/Houssam/ObjectDetection/cityscapes_yolo/images/val

nc: 10
names: ['person', 'rider', 'car', 'truck', 'bus', 'motorcycle', 'bicycle', 'train', 'traffic light', 'traffic sign']



#### 🏁 9️⃣ Train YOLOv5 on the Cityscapes Dataset

##### 📦 9.1️⃣ Clone YOLOv5 and Install Dependencies


In [12]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


d:\Houssam\ObjectDetection\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


Defaulting to user installation because normal site-packages is not writeable


##### 🧠 9.2️⃣ Train YOLOv5 on Cityscapes

In [ ]:
# 🏋️‍♂️ 9️⃣ Train YOLOv8 Model Using Python API



# Load a pretrained YOLOv8 model (you can switch to yolov8n.pt or yolov8m.pt based on resources)
model = YOLO('yolov8s.pt')

# Train the model
model.train(
    data='D:/Houssam/ObjectDetection/cityscapes_yolo/data.yaml',  # path to your YAML
    epochs=30,                     # number of epochs
    imgsz=640,                     # image size
    batch=8,                       # adjust based on your system
    project='cityscapes_yolo',    # where to save runs
    name='yolov8s_results',       # folder name for this training run
    exist_ok=True                 # don't raise error if folder exists
)


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\LENOVO\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 21.5M/21.5M [00:59<00:00, 377kB/s]


Ultralytics 8.3.156  Python-3.11.8 torch-2.7.1+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Houssam/ObjectDetection/cityscapes_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_results, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=

100%|██████████| 755k/755k [00:02<00:00, 374kB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytic

...

Plotting labels to cityscapes_yolo\yolov8s_results\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to cityscapes_yolo\yolov8s_results
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/30         0G      1.377      1.274      1.034        393        640: 100%|██████████| 372/372 [51:49<00:00,  8.36s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [02:11<00:00,  4.10s/it]

                   all        500      15937      0.466      0.356      0.361      0.206



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/30         0G      1.352     0.9593      1.017        411        640:  80%|████████  | 299/372 [44:15<10:55,  8.99s/it] 

In [ ]:
from ultralytics import YOLO

model = YOLO('D:/Houssam/ObjectDetection/cityscapes_yolo/yolov8s_results/weights/last.pt')
model.train(resume=True)


Ultralytics 8.3.156  Python-3.11.8 torch-2.7.1+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Houssam/ObjectDetection/cityscapes_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=D:\Houssam\ObjectDetection\cityscapes_yolo\yolov8s_results\weights\last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s_results, nbs=64, nms=False, opset=None, optimize=Fals

train: Scanning D:\Houssam\ObjectDetection\cityscapes_yolo\labels\train.cache... 2974 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]

train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000000_000019_leftImg8bit.png: 5 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000002_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000022_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000030_000019_leftImg8bit.png: 4 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000031_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000042_000019_leftImg8bit.png: 2 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000054_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000060_000019_leftImg8bit.png: 15 


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.60.3 ms, read: 39.23.4 MB/s, size: 2253.1 KB)


...

                   all        500      15937      0.554      0.392      0.428      0.246



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      11/30         0G      1.242     0.7891     0.9844        474        640: 100%|██████████| 372/372 [2:20:00<00:00, 22.58s/it]     
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [02:42<00:00,  5.07s/it]


                   all        500      15937      0.627      0.414      0.453      0.259

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      12/30         0G      1.231     0.7772     0.9743        334        640:  14%|█▎        | 51/372 [08:44<59:27, 11.11s/it]  

In [1]:
from ultralytics import YOLO

model = YOLO('D:/Houssam/ObjectDetection/cityscapes_yolo/yolov8s_results/weights/last.pt')
model.train(resume=True)


New https://pypi.org/project/ultralytics/8.3.158 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.156  Python-3.11.8 torch-2.7.1+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Houssam/ObjectDetection/cityscapes_yolo/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=D:\Houssam\ObjectDetection\cityscapes_yolo\yolov8s_results\weights\last.pt, momentum=0.9

train: Scanning D:\Houssam\ObjectDetection\cityscapes_yolo\labels\train.cache... 2974 images, 1 backgrounds, 0 corrupt: 100%|██████████| 2975/2975 [00:00<?, ?it/s]

train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000000_000019_leftImg8bit.png: 5 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000002_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000022_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000030_000019_leftImg8bit.png: 4 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000031_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000042_000019_leftImg8bit.png: 2 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000054_000019_leftImg8bit.png: 1 duplicate labels removed
train: D:\Houssam\ObjectDetection\cityscapes_yolo\images\train\aachen_000060_000019_leftImg8bit.png: 15 


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 6.59.0 ms, read: 59.68.9 MB/s, size: 2253.1 KB)


...

                   all        500      15937       0.69      0.436      0.492      0.294



7 epochs completed in 7.400 hours.
Optimizer stripped from cityscapes_yolo\yolov8s_results\weights\last.pt, 22.5MB
Optimizer stripped from cityscapes_yolo\yolov8s_results\weights\best.pt, 22.5MB

Validating cityscapes_yolo\yolov8s_results\weights\best.pt...
Ultralytics 8.3.156  Python-3.11.8 torch-2.7.1+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [02:02<00:00,  3.84s/it]


                   all        500      15937       0.69      0.436      0.492      0.294
                person        402       3415      0.727      0.462      0.545      0.307
                 rider        254        552      0.735      0.478       0.57      0.331
                   car        478       4667      0.831      0.646      0.731      0.505
                 truck         80         93      0.654      0.366      0.401        0.3
                   bus         75         98      0.799      0.541      0.631      0.475
            motorcycle         93        149      0.599      0.356      0.392      0.184
               bicycle        331       1171      0.717      0.357      0.457      0.234
                 train         22         23      0.509      0.522      0.465      0.237
         traffic light        291       1661      0.627      0.312      0.344      0.153
          traffic sign        473       4108      0.701      0.324      0.383      0.213
Speed: 2.5ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001F391F38B10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,   

🧪 1️⃣ Evaluate the Trained Model (Validation Results)

In [2]:
model = YOLO('cityscapes_yolo/yolov8s_results/weights/last.pt')

In [3]:
# 📊 Evaluate the model on the validation set
from ultralytics import YOLO

model = YOLO('cityscapes_yolo/yolov8s_results/weights/last.pt')  # or last.pt
metrics = model.val()  # this runs evaluation on val set

Ultralytics 8.3.156  Python-3.11.8 torch-2.7.1+cpu CPU (Intel Core(TM) i5-1035G1 1.00GHz)
Model summary (fused): 72 layers, 11,129,454 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 11.716.9 ms, read: 48.14.7 MB/s, size: 2321.1 KB)


val: Scanning D:\Houssam\ObjectDetection\cityscapes_yolo\labels\val.cache... 497 images, 3 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]

val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\frankfurt_000001_016273_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\frankfurt_000001_017101_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\frankfurt_000001_077434_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\munster_000026_000019_leftImg8bit.png: 2 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\munster_000029_000019_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\munster_000049_000019_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\munster_000108_000019_leftImg8bit.png: 1 duplicate labels removed
val: D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\munster_000140_000019_leftImg8bit.png: 1 duplicate labels re


c:\Users\LENOVO\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [01:54<00:00,  3.57s/it]


                   all        500      15937       0.69      0.436      0.492      0.294
                person        402       3415      0.727      0.462      0.545      0.307
                 rider        254        552      0.735      0.478       0.57      0.331
                   car        478       4667      0.831      0.646      0.731      0.505
                 truck         80         93      0.654      0.366      0.401        0.3
                   bus         75         98      0.799      0.541      0.631      0.475
            motorcycle         93        149      0.599      0.356      0.392      0.184
               bicycle        331       1171      0.717      0.357      0.457      0.234
                 train         22         23      0.509      0.522      0.465      0.237
         traffic light        291       1661      0.627      0.312      0.344      0.153
          traffic sign        473       4108      0.701      0.324      0.383      0.213
Speed: 0.9ms preproce

In [3]:
results = model('D:/Houssam/ObjectDetection/cityscapes_yolo/images/val/frankfurt_000001_016273_leftImg8bit.png', save=False)

# Define output directory
output_dir = 'D:/Houssam/ObjectDetection/predicted_images'
os.makedirs(output_dir, exist_ok=True)

# Save image with smaller labels
for i, r in enumerate(results):
    im_array = r.plot(line_width=1, font_size=0.9)  # Smaller boxes and text
    output_path = os.path.join(output_dir, f'prediction_{i}.jpg')
    cv2.imwrite(output_path, im_array)
    print(f"✅ Image saved to {output_path}")


image 1/1 D:\Houssam\ObjectDetection\cityscapes_yolo\images\val\frankfurt_000001_016273_leftImg8bit.png: 320x640 4 persons, 3 cars, 1 train, 3 traffic lights, 4 traffic signs, 303.6ms
Speed: 9.3ms preprocess, 303.6ms inference, 12.4ms postprocess per image at shape (1, 3, 320, 640)
✅ Image saved to D:/Houssam/ObjectDetection/predicted_images\prediction_0.jpg


Test it on a new image

In [4]:
def run_prediction(image_path, output_name):
    if not os.path.exists(image_path):
        print(f"❌ File not found: {image_path}")
        return

    results = model(image_path, save=False)

    for i, r in enumerate(results):
        im_array = r.plot(line_width=1, font_size=0.7)  # Small text
        output_path = os.path.join(output_dir, output_name)
        cv2.imwrite(output_path, im_array)
        print(f"✅ Prediction saved to: {output_path}")

# Paths
img_path1 = 'D:/Houssam/ObjectDetection/custom_tests/test_image.jpg'
img_path2 = 'D:/Houssam/ObjectDetection/custom_tests/test_image2.jpg'
img_path3 = 'D:/Houssam/ObjectDetection/custom_tests/img.jpg'
img_path4 = 'D:/Houssam/ObjectDetection/custom_tests/imgg.jpg'
img_path5 = 'D:/Houssam/ObjectDetection/custom_tests/1.jpg'
output_dir = 'D:/Houssam/ObjectDetection/predicted_images'
os.makedirs(output_dir, exist_ok=True)

# Run predictions
run_prediction(img_path1, 'custom_test_1.jpg')
run_prediction(img_path2, 'custom_test_2.jpg')
run_prediction(img_path3, 'custom_test_3.jpg')
run_prediction(img_path4, 'custom_test_4.jpg')
run_prediction(img_path5, 'custom_test_5.jpg')


image 1/1 D:\Houssam\ObjectDetection\custom_tests\test_image.jpg: 384x640 4 persons, 9 cars, 242.4ms
Speed: 6.4ms preprocess, 242.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)
✅ Prediction saved to: D:/Houssam/ObjectDetection/predicted_images\custom_test_1.jpg

image 1/1 D:\Houssam\ObjectDetection\custom_tests\test_image2.jpg: 448x640 (no detections), 174.6ms
Speed: 4.2ms preprocess, 174.6ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)
✅ Prediction saved to: D:/Houssam/ObjectDetection/predicted_images\custom_test_2.jpg

image 1/1 D:\Houssam\ObjectDetection\custom_tests\img.jpg: 480x640 6 traffic signs, 185.8ms
Speed: 2.2ms preprocess, 185.8ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
✅ Prediction saved to: D:/Houssam/ObjectDetection/predicted_images\custom_test_3.jpg

image 1/1 D:\Houssam\ObjectDetection\custom_tests\imgg.jpg: 448x640 (no detections), 159.8ms
Speed: 2.6ms preprocess, 159.8ms inference, 0.6ms postprocess

bytetrack_demo.py

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

class KalmanBoxTracker:
    count = 0
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1, 0, 0, 0, 1, 0, 0],
                              [0, 1, 0, 0, 0, 1, 0],
                              [0, 0, 1, 0, 0, 0, 1],
                              [0, 0, 0, 1, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 0, 0, 1]])
        self.kf.H = np.array([[1, 0, 0, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 1, 0, 0, 0, 0],
                              [0, 0, 0, 1, 0, 0, 0]])
        self.kf.R[2:, 2:] *= 10.
        self.kf.P[4:, 4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1, -1] *= 0.01
        self.kf.Q[4:, 4:] *= 0.01
        self.kf.x[:4] = self._convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1

    def update(self, bbox):
        self.time_since_update = 0
        self.kf.update(self._convert_bbox_to_z(bbox))

    def predict(self):
        self.kf.predict()
        self.time_since_update += 1
        return self._convert_x_to_bbox(self.kf.x)

    def get_state(self):
        return self._convert_x_to_bbox(self.kf.x)

    @staticmethod
    def _convert_bbox_to_z(bbox):
        x1, y1, x2, y2 = bbox
        w = x2 - x1
        h = y2 - y1
        x = x1 + w / 2.
        y = y1 + h / 2.
        s = w * h
        r = w / float(h)
        return np.array([[x], [y], [s], [r]])

    @staticmethod
    def _convert_x_to_bbox(x):
        w = np.sqrt(x[2] * x[3])
        h = x[2] / w
        x1 = x[0] - w / 2.
        y1 = x[1] - h / 2.
        x2 = x[0] + w / 2.
        y2 = x[1] + h / 2.
        return [x1[0], y1[0], x2[0], y2[0]]

class KalmanTrackerManager:
    def __init__(self, iou_thresh=0.3, max_age=30):
        self.trackers = []
        self.iou_thresh = iou_thresh
        self.max_age = max_age
        self.frame_count = 0

    def update(self, detections):
        self.frame_count += 1
        updated_tracks = []
        # Predict all trackers
        for tracker in self.trackers:
            tracker.predict()
        unmatched_dets = []
        for i, det in enumerate(detections):
            matched = False
            for tracker in self.trackers:
                iou = self._iou(det[:4], tracker.get_state())
                if iou >= self.iou_thresh:
                    tracker.update(det[:4])
                    updated_tracks.append((*tracker.get_state(), tracker.id, det[5]))
                    matched = True
                    break
            if not matched:
                unmatched_dets.append(det)
        # Create new trackers for unmatched detections
        for det in unmatched_dets:
            tracker = KalmanBoxTracker(det[:4])
            updated_tracks.append((*tracker.get_state(), tracker.id, det[5]))
            self.trackers.append(tracker)
        # Clean up old trackers
        self.trackers = [t for t in self.trackers if t.time_since_update <= self.max_age]
        return updated_tracks

    @staticmethod
    def _iou(bb1, bb2):
        x1 = max(bb1[0], bb2[0])
        y1 = max(bb1[1], bb2[1])
        x2 = min(bb1[2], bb2[2])
        y2 = min(bb1[3], bb2[3])
        inter_area = max(0, x2 - x1) * max(0, y2 - y1)
        bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
        bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
        return inter_area / float(bb1_area + bb2_area - inter_area + 1e-6)

# Load YOLOv8 model
model = YOLO("cityscapes_yolo/yolov8s_results/weights/last.pt")
tracker = KalmanTrackerManager()

# Video I/O
cap = cv2.VideoCapture("vid1.mp4") # Ensure 'vid.mp4' exists or provide full path
output_path = "tracking_output.mp4" # Original output name

# Get video properties for output writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define class names for visualization (based on your Cityscapes mapping)
class_names = {
    0: "person", 1: "rider", 2: "car", 3: "truck", 4: "bus",
    5: "motorcycle", 6: "bicycle", 7: "train", 8: "traffic light", 9: "traffic sign"
}

# Define a color map for each class (BGR format for OpenCV)
# You can customize these colors as you like!
class_colors = {
    0: (0, 255, 255),   # Yellow for person
    1: (255, 0, 255),   # Magenta for rider
    2: (255, 0, 0),     # Blue for car
    3: (0, 0, 255),     # Red for truck
    4: (0, 255, 0),     # Green for bus
    5: (255, 255, 0),   # Cyan for motorcycle
    6: (128, 0, 128),   # Purple for bicycle
    7: (0, 128, 255),   # Orange for train
    8: (0, 165, 255),   # Orange-Red for traffic light
    9: (255, 128, 0)    # Light Blue for traffic sign
}


try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False)[0]
        detections = [
            [*box.xyxy[0].tolist(), box.conf.item(), box.cls.item()]
            for box in results.boxes
        ]
        
        tracked = tracker.update(detections)
        
        for x1, y1, x2, y2, obj_id, cls_id in tracked:
            # Get the color for the current class ID
            color = class_colors.get(int(cls_id), (255, 255, 255)) # Default to white if class ID not found

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Get class name
            class_name = class_names.get(int(cls_id), "unknown")
            
            # Display ID and Class Name with the assigned color
            label = f'ID:{obj_id} {class_name}'
            cv2.putText(frame, label, (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        out.write(frame)

finally:
    cap.release()
    out.release()
    print(f"✅ Tracking completed. Saved to: {output_path}")


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13528\2934668985.py:56: RuntimeWarning: invalid value encountered in sqrt
  w = np.sqrt(x[2] * x[3])


✅ Tracking completed. Saved to: tracking_output.mp4


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

class KalmanBoxTracker:
    count = 0
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1, 0, 0, 0, 1, 0, 0],
                              [0, 1, 0, 0, 0, 1, 0],
                              [0, 0, 1, 0, 0, 0, 1],
                              [0, 0, 0, 1, 0, 0, 0],
                              [0, 0, 0, 0, 1, 0, 0],
                              [0, 0, 0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 0, 0, 1]])
        self.kf.H = np.array([[1, 0, 0, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0, 0, 0],
                              [0, 0, 1, 0, 0, 0, 0],
                              [0, 0, 0, 1, 0, 0, 0]])
        self.kf.R[2:, 2:] *= 10.
        self.kf.P[4:, 4:] *= 1000.
        self.kf.P *= 10.
        self.kf.Q[-1, -1] *= 0.01
        self.kf.Q[4:, 4:] *= 0.01
        self.kf.x[:4] = self._convert_bbox_to_z(bbox)
        self.time_since_update = 0
        self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1

    def update(self, bbox):
        self.time_since_update = 0
        self.kf.update(self._convert_bbox_to_z(bbox))

    def predict(self):
        self.kf.predict()
        self.time_since_update += 1
        return self._convert_x_to_bbox(self.kf.x)

    def get_state(self):
        return self._convert_x_to_bbox(self.kf.x)

    @staticmethod
    def _convert_bbox_to_z(bbox):
        x1, y1, x2, y2 = bbox
        w = x2 - x1
        h = y2 - y1
        x = x1 + w / 2.
        y = y1 + h / 2.
        s = w * h
        r = w / float(h)
        return np.array([[x], [y], [s], [r]])

    @staticmethod
    def _convert_x_to_bbox(x):
        w = np.sqrt(x[2] * x[3])
        h = x[2] / w
        x1 = x[0] - w / 2.
        y1 = x[1] - h / 2.
        x2 = x[0] + w / 2.
        y2 = x[1] + h / 2.
        return [x1[0], y1[0], x2[0], y2[0]]

class KalmanTrackerManager:
    def __init__(self, iou_thresh=0.5, max_age=30):
        self.trackers = []
        self.iou_thresh = iou_thresh
        self.max_age = max_age
        self.frame_count = 0

    def update(self, detections):
        self.frame_count += 1
        updated_tracks = []
        # Predict all trackers
        for tracker in self.trackers:
            tracker.predict()
        unmatched_dets = []
        for i, det in enumerate(detections):
            matched = False
            for tracker in self.trackers:
                iou = self._iou(det[:4], tracker.get_state())
                if iou >= self.iou_thresh:
                    tracker.update(det[:4])
                    updated_tracks.append((*tracker.get_state(), tracker.id, det[5]))
                    matched = True
                    break
            if not matched:
                unmatched_dets.append(det)
        # Create new trackers for unmatched detections
        for det in unmatched_dets:
            tracker = KalmanBoxTracker(det[:4])
            updated_tracks.append((*tracker.get_state(), tracker.id, det[5]))
            self.trackers.append(tracker)
        # Clean up old trackers
        self.trackers = [t for t in self.trackers if t.time_since_update <= self.max_age]
        return updated_tracks

    @staticmethod
    def _iou(bb1, bb2):
        x1 = max(bb1[0], bb2[0])
        y1 = max(bb1[1], bb2[1])
        x2 = min(bb1[2], bb2[2])
        y2 = min(bb1[3], bb2[3])
        inter_area = max(0, x2 - x1) * max(0, y2 - y1)
        bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
        bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])
        return inter_area / float(bb1_area + bb2_area - inter_area + 1e-6)

# Load YOLOv8 model
model = YOLO("cityscapes_yolo/yolov8s_results/weights/last.pt")
tracker = KalmanTrackerManager()

# Video I/O
cap = cv2.VideoCapture("vid2.mp4") # Ensure 'vid.mp4' exists or provide full path
output_path = "tracking_output_4.mp4" # Original output name

# Get video properties for output writer
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*"mp4v")

out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Define a horizontal counting line (e.g., at 60% of the frame height)
counting_line_y = int(frame_height * 0.6)

# Initialize counters
up_count = 0
down_count = 0

# Store the previous centroid y-coordinate for each track to detect crossings
track_previous_y = {}

# Define class names for visualization (based on your Cityscapes mapping)
class_names = {
    0: "person", 1: "rider", 2: "car", 3: "truck", 4: "bus",
    5: "motorcycle", 6: "bicycle", 7: "train", 8: "traffic light", 9: "traffic sign"
}

# Define a color map for each class (BGR format for OpenCV)
class_colors = {
    0: (0, 255, 255),   # Yellow for person
    1: (255, 0, 255),   # Magenta for rider
    2: (255, 0, 0),     # Blue for car
    3: (0, 0, 255),     # Red for truck
    4: (0, 255, 0),     # Green for bus
    5: (255, 255, 0),   # Cyan for motorcycle
    6: (128, 0, 128),   # Purple for bicycle
    7: (0, 128, 255),   # Orange for train
    8: (0, 165, 255),   # Orange-Red for traffic light
    9: (255, 128, 0)    # Light Blue for traffic sign
}

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose=False)[0]
        detections = [
            [*box.xyxy[0].tolist(), box.conf.item(), box.cls.item()]
            for box in results.boxes
        ]
        
        tracked = tracker.update(detections)

        # Process tracked objects for counting
        current_active_ids = set()
        for x1, y1, x2, y2, obj_id, cls_id in tracked:
            current_active_ids.add(obj_id)

            # Calculate current centroid y-coordinate
            current_cy = (y1 + y2) / 2

            # Get previous centroid y-coordinate
            previous_cy = track_previous_y.get(obj_id)

            if previous_cy is not None:
                # Check for crossing the line
                if previous_cy < counting_line_y and current_cy >= counting_line_y:
                    down_count += 1
                    print(f"Object ID {obj_id} crossed DOWN. Total down: {down_count}")
                elif previous_cy > counting_line_y and current_cy <= counting_line_y:
                    up_count += 1
                    print(f"Object ID {obj_id} crossed UP. Total up: {up_count}")
            
            # Update the previous y-coordinate for the next frame
            track_previous_y[obj_id] = current_cy

        # Clean up track_previous_y for objects that are no longer tracked
        keys_to_delete = [obj_id for obj_id in track_previous_y if obj_id not in current_active_ids]
        for obj_id in keys_to_delete:
            del track_previous_y[obj_id]
        
        for x1, y1, x2, y2, obj_id, cls_id in tracked:
            # Get the color for the current class ID
            color = class_colors.get(int(cls_id), (255, 255, 255)) # Default to white if class ID not found

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            
            # Get class name
            class_name = class_names.get(int(cls_id), "unknown")
            
            # Display ID and Class Name with the assigned color
            label = f'ID:{obj_id} {class_name}'
            cv2.putText(frame, label, (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

        # Draw the counting line
        cv2.line(frame, (0, counting_line_y), (frame_width, counting_line_y), (0, 255, 255), 2) # Yellow line

        # Display counts
        cv2.putText(frame, f'Up: {up_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
        cv2.putText(frame, f'Down: {down_count}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        out.write(frame)

finally:
    cap.release()
    out.release()
    print(f"✅ Tracking completed. Saved to: {output_path}")


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_13528\2586286573.py:56: RuntimeWarning: invalid value encountered in sqrt
  w = np.sqrt(x[2] * x[3])


Object ID 48 crossed DOWN. Total down: 1
Object ID 256 crossed UP. Total up: 1
✅ Tracking completed. Saved to: tracking_output_4.mp4
